### --- Day 7: Bridge Repair ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2024/day/7

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines = LoadPuzzleInput(2024, 7);
WriteLines(inputLines);

Loading puzzle file: Day7.txt
Total lines: 850
Max line length: 41

194558: 2 6 664 40 874 40 7
127536599: 49 4 21 65 99
26009943184: 6 6 83 31 25 72 5 4 39 4
250121527725: 8 8 1 32 907 4 7 7 2 4 7 4
5493508: 3 893 785 38 86


In [4]:
string[] testInputLines = [
    "190: 10 19",
    "3267: 81 40 27",
    "83: 17 5",
    "156: 15 6",
    "7290: 6 8 6 15",
    "161011: 16 10 13",
    "192: 17 8 14",
    "21037: 9 7 18 13",
    "292: 11 6 16 20",
];

In [5]:
using Instr = System.Func<long, long, long>;

long Add(long a, long b) => a + b;
long Mult(long a, long b) => a * b;

Instr[] part1Instructions = [Add, Mult]; // spoiler alert!

In [6]:
// I think we're going to need this. Combine all items from source into an array

IEnumerable<IEnumerable<T>> Combine<T>(IEnumerable<T> source, int length) => length switch
{
    1 => source.Select(item => new T[] { item }),
    _ => source.SelectMany(item => Combine(source, length - 1).Select(rest => rest.Append(item))),
};

In [ ]:
// Just for fun, let's try a non-recursive option

IEnumerable<IEnumerable<T>> CombineAlt<T>(IList<T> source, int length)
{
    // Let's do this in the style of an "adder", i.e., increment all values for
    // the lowest digit, and when it increments past the last value, reset and
    // "carry the one", i.e., increment the next digit.

    // Retrospective note: turns out this function does seem to be faster than
    // the original function, presumably because there is less stack traversal and.
    // array allocation.

    int[] adder = new int[length];
    T[] buffer = adder.Select(i => source[i]).ToArray();

    bool Next()
    {
        var i = 0;
        while (i < length)
        {
            adder[i]++;
            buffer[i] = source[adder[i] % source.Count];
            if (adder[i] == source.Count)
            {
                // "Carry the one"
                adder[i] = 0;
                i++;
            } 
            else 
            {
                // Still enumerating the current digit
                return true;
            }
        }

        // We have "overflowed" ie no more combos to enumerate
        return false;
    }

    do
    {
        yield return buffer;
    } while (Next());
}

In [8]:
long Compute(string row) => ComputeBase(row, part1Instructions);

long ComputeBase(string row, Instr[] instructions)
{
    var rowBits = row.Split([':', ' '], StringSplitOptions.RemoveEmptyEntries).Select(long.Parse).ToArray();
    var target = rowBits[0];
    var inputs = rowBits[1..];

    var mathCombos = Combine(instructions, inputs.Length - 1).Select(c => c.ToArray());
    foreach (var mathCombo in mathCombos)
    {
        var i = 0;
        var result = inputs.Aggregate((a, b) => mathCombo[i++](a, b));
        if (result == target) {
            return target;
        }
    }

    return 0;
}

In [9]:
// In the above example, the sum of the test values for the three equations listed above is 3749.

var testAnswer = testInputLines.Select(Compute).Sum();
Console.WriteLine(testAnswer);

3749


In [10]:
// Determine which equations could possibly be true. What is their total calibration result?

var part1Answer = inputLines.Select(Compute).Sum();
Console.WriteLine(part1Answer);

5512534574980


In [11]:
// 5512534574980 is correct!
Ensure(5512534574980, part1Answer);

### --- Part Two ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2024/day/7

In [13]:
long Concat(long a, long b) => long.Parse($"{a}{b}");

Instr[] part2Instructions = [..part1Instructions, Concat];

long Part2Compute(string row) => ComputeBase(row, part2Instructions);

In [14]:
// Adding up all six test values ... produces the new total calibration result of 11387.

var part2TestAnswer = testInputLines.Select(Part2Compute).Sum();
Console.WriteLine(part2TestAnswer);

11387


In [15]:
var part2Answer = inputLines.Select(Part2Compute).Sum();
Console.WriteLine(part2Answer);

328790210468594


In [16]:
// 328790210468594 is correct!
Ensure(328790210468594, part2Answer);